In [98]:
import os
import requests
from dotenv import load_dotenv
from unstract.llmwhisperer import LLMWhispererClientV2
from google import genai
from pydantic import BaseModel

In [99]:
from typing import List, Optional

In [100]:
class LineItems(BaseModel):
    name: str
    cost: List[str]


class Reciepts(BaseModel):
    image_url: Optional[str]
    date: Optional[str]
    shop_name: Optional[str]
    total_amount: Optional[str]
    line_items: List[LineItems]

In [101]:
load_dotenv()

LLMWHISPERER_API_KEY = os.getenv("LLMWHISPERER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
MONGO_URL = os.getenv("MONGO_URL")

In [102]:
def download_image(url, folder="bills", filename="downloaded_image.jpg"):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Get the image data
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        image_path = os.path.join(folder, filename)
        with open(image_path, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Image downloaded successfully: {image_path}")
    else:
        print("Failed to download image. Check the URL.")

In [103]:
def convert_receipt_to_text(image_path: str):
    clientllm = LLMWhispererClientV2(
        base_url="https://llmwhisperer-api.us-central.unstract.com/api/v2",
        api_key=LLMWHISPERER_API_KEY,
        logging_level="INFO",
    )
    whisper = clientllm.whisper(
        file_path=image_path, wait_for_completion=True, wait_timeout=200
    )
    output = whisper["extraction"]["result_text"]
    return output

In [104]:
def convert_text_to_json(output: str):
    client = genai.Client(api_key=GEMINI_API_KEY)
    data = "convert this to json." + output
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=data,
        config={
            "response_mime_type": "application/json",
            "response_schema": Reciepts,
        },
    )
    final_lineitems = response.parsed
    print("respone", response.text)
    print(final_lineitems)
    return final_lineitems

In [105]:
image_url = "https://media.istockphoto.com/id/1420767944/vector/register-sale-receipt-isolated-on-white-background-cash-receipt-printed.jpg?s=612x612&w=0&k=20&c=eV7CDJK0DZgKo7KVlGTDJeVMN_2xybqIPvt1ATl_kkM="

In [106]:
download_image(image_url)
image_path = "./bills/downloaded_image.jpg"
output = convert_receipt_to_text(image_path)
final_lineitems = convert_text_to_json(output)

Image downloaded successfully: bills/downloaded_image.jpg
respone {
  "image_url": null,
  "date": "MM/DD/YYYY",
  "shop_name": "COMPANY NAME",
  "total_amount": "$29.69",
  "line_items": [
    {
      "name": "Orange Juice",
      "cost": [
        "$2.15"
      ]
    },
    {
      "name": "Apples",
      "cost": [
        "$3.50"
      ]
    },
    {
      "name": "Tomato",
      "cost": [
        "$2.40"
      ]
    },
    {
      "name": "Fish",
      "cost": [
        "$6.99"
      ]
    },
    {
      "name": "Beef",
      "cost": [
        "$10.00"
      ]
    },
    {
      "name": "Onion",
      "cost": [
        "$1.25"
      ]
    },
    {
      "name": "Cheese",
      "cost": [
        "$3.40"
      ]
    }
  ]
}
image_url=None date='MM/DD/YYYY' shop_name='COMPANY NAME' total_amount='$29.69' line_items=[LineItems(name='Orange Juice', cost=['$2.15']), LineItems(name='Apples', cost=['$3.50']), LineItems(name='Tomato', cost=['$2.40']), LineItems(name='Fish', cost=['$6.99']), 

In [107]:
# this is when final_lineitems =  response.text
# with open("output.json", "w") as file:
#     file.write(final_lineitems)
# final_lineitems

In [108]:
# final = json.loads(final_lineitems)
# print(final)
# Reciepts(**final)

In [109]:
Reciepts.model_validate(final_lineitems)

Reciepts(image_url=None, date='MM/DD/YYYY', shop_name='COMPANY NAME', total_amount='$29.69', line_items=[LineItems(name='Orange Juice', cost=['$2.15']), LineItems(name='Apples', cost=['$3.50']), LineItems(name='Tomato', cost=['$2.40']), LineItems(name='Fish', cost=['$6.99']), LineItems(name='Beef', cost=['$10.00']), LineItems(name='Onion', cost=['$1.25']), LineItems(name='Cheese', cost=['$3.40'])])